# 🗑️ VLM для распознавания мусора (Ансамбль + LLM)

**Архитектура:**
```
Изображение → Ваш Ансамбль (YOLO+RT-DETR) → Детекции → LLM → Ответ
```

Этот подход использует ваши обученные детекторы как "глаза" VLM.


In [ ]:
# 1. Установка и клонирование
!git lfs install
!git clone https://github.com/sry4theweight/VLM_garbage.git
%cd VLM_garbage

# Установка зависимостей
!pip install -q transformers peft accelerate ultralytics supervision tqdm pillow roboflow

# Проверка GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# 2. Скачать датасет Roboflow
!python download_roboflow_dataset.py


In [ ]:
# 3. Загрузка ансамбля и VLM
import sys
sys.path.append('.')

from vlm_annotation.ensemble_detector import EnsembleDetector
from vlm_inference.ensemble_vlm import EnsembleVLM
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import random

# Загрузка VLM (Ансамбль + LLM)
vlm = EnsembleVLM(
    yolo_model_path="models/yolo/yolov8x/best.pt",
    detr_model_path="models/rt-detr/rt-detr-101/m",
    detr_processor_path="models/rt-detr/rt-detr-101/p",
    llm_model="microsoft/phi-2",
    conf_threshold=0.5
)
print("✅ EnsembleVLM готов!")


In [ ]:
# 4. Собираем изображения БЕЗ АУГМЕНТАЦИЙ (valid + test)
valid_dir = Path("data/roboflow_dataset/valid")
test_dir = Path("data/roboflow_dataset/test")

all_images = list(valid_dir.glob("*.jpg")) + list(test_dir.glob("*.jpg"))
print(f"✅ Найдено {len(all_images)} изображений (valid + test, без аугментаций)")


In [ ]:
# 5. Функция для тестирования VLM
COLORS = {'glass': 'green', 'plastic': 'red', 'metal': 'cyan', 'paper': 'yellow', 'organic': 'lime'}

def test_vlm_on_image(image_path, questions):
    print(f"\n{'='*60}")
    print(f"📷 {Path(image_path).name}")
    print('='*60)
    
    # Показываем изображение с детекциями
    img = Image.open(image_path).convert('RGB')
    detections = vlm.detector.detect(img)
    
    fig, ax = plt.subplots(1, figsize=(10, 7))
    ax.imshow(img)
    
    for det in detections:
        bbox = det['box']
        color = COLORS.get(det['label'], 'white')
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1],
                                  linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        ax.text(bbox[0], bbox[1]-5, f"{det['label']} {det['confidence']:.0%}", 
                color=color, fontsize=9, weight='bold')
    
    ax.set_title(f"Детекции: {len(detections)} объектов")
    ax.axis('off')
    plt.show()
    
    # Отвечаем на вопросы
    print("\n💬 Вопросы и ответы:")
    for q in questions:
        answer = vlm.answer_question(str(image_path), q)
        print(f"\nQ: {q}")
        print(f"A: {answer}")


In [ ]:
# 6. Тестируем на 3 случайных изображениях
questions = [
    "What objects are in this image?",
    "Is there any plastic in this image?",
    "How many objects are there?",
]

for img_path in random.sample(all_images, min(3, len(all_images))):
    test_vlm_on_image(img_path, questions)


In [ ]:
# 7. Интерактивное тестирование - измените вопрос!
image_path = random.choice(all_images)
my_question = "Is there any glass in this image?"  # <-- Ваш вопрос

test_vlm_on_image(image_path, [my_question])


In [ ]:
# 8. Сохранение на Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/VLM_garbage
!cp -r models /content/drive/MyDrive/VLM_garbage/
!cp -r vlm_inference /content/drive/MyDrive/VLM_garbage/
!cp -r vlm_annotation /content/drive/MyDrive/VLM_garbage/

print("✅ Сохранено на Google Drive: /MyDrive/VLM_garbage")
